In [3]:
import pandas as pd

In [14]:
import numpy as np

In [15]:

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [5]:
df = pd.read_csv('steam.csv')

In [6]:
df.head()

appid                       name release_date  english         developer  \
0     10             Counter-Strike   2000-11-01        1             Valve   
1     20      Team Fortress Classic   1999-04-01        1             Valve   
2     30              Day of Defeat   2003-05-01        1             Valve   
3     40         Deathmatch Classic   2001-06-01        1             Valve   
4     50  Half-Life: Opposing Force   1999-11-01        1  Gearbox Software   

  publisher          platforms  required_age  \
0     Valve  windows;mac;linux             0   
1     Valve  windows;mac;linux             0   
2     Valve  windows;mac;linux             0   
3     Valve  windows;mac;linux             0   
4     Valve  windows;mac;linux             0   

                                          categories  genres  \
0  Multi-player;Online Multi-Player;Local Multi-P...  Action   
1  Multi-player;Online Multi-Player;Local Multi-P...  Action   
2              Multi-player;Valve Anti-Cheat enabled  Action   
3  Multi-player;Online Multi-Player;Local Multi-P...  Action   
4  Single-player;Multi-player;Valve Anti-Cheat en...  Action   

                  steamspy_tags  achievements  positive_ratings  \
0        Action;FPS;Multiplayer             0            124534   
1        Action;FPS;Multiplayer             0              3318   
2  FPS;World War II;Multiplayer             0              3416   
3        Action;FPS;Multiplayer             0              1273   
4             FPS;Action;Sci-fi             0              5250   

   negative_ratings  average_playtime  median_playtime             owners  \
0              3339             17612              317  10000000-20000000   
1               633               277               62   5000000-10000000   
2               398               187               34   5000000-10000000   
3               267               258              184   5000000-10000000   
4               288               624              415   5000000-10000000   

   price  
0   7.19  
1   3.99  
2   3.99  
3   3.99  
4   3.99

In [8]:
df['score'] = df['positive_ratings'] / (df['positive_ratings'] + df['negative_ratings'])

In [9]:
df['label'] = df['score'].apply(lambda x: 1 if x > 0.8 else 0)  # 1 = Bom, 0 = Ruim

In [10]:
df = df[['price', 'release_date', 'developer', 'positive_ratings', 'negative_ratings']].dropna()
df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year
df = df.dropna()

In [11]:
df['score'] = df['positive_ratings'] / (df['positive_ratings'] + df['negative_ratings'])
df['label'] = df['score'].apply(lambda x: 1 if x > 0.8 else 0)

In [12]:
df['developer'] = df['developer'].astype('category').cat.codes

In [13]:
X = df[['price', 'release_year', 'developer', 'positive_ratings', 'negative_ratings']]
y = df['label']

In [16]:
# Selecionar colunas importantes
df = df[['price', 'release_date', 'developer', 'positive_ratings', 'negative_ratings']]

# Remover valores ausentes
df = df.dropna()

# Converter release_date em ano
df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year
df = df.dropna(subset=['release_year'])

# Converter 'developer' para números (codificação categórica)
df['developer'] = df['developer'].astype('category').cat.codes

# Criar uma variável de score com base em avaliações positivas
df['score'] = df['positive_ratings'] / (df['positive_ratings'] + df['negative_ratings'])

# Criar a classificação binária: Bom (score > 0.8) ou Ruim
df['label'] = df['score'].apply(lambda x: 1 if x > 0.8 else 0)

# Visualizar
df[['price', 'release_year', 'developer', 'positive_ratings', 'negative_ratings', 'label']].head()


price  release_year  developer  positive_ratings  negative_ratings  label
0   7.19          2000      15117            124534              3339      1
1   3.99          1999      15117              3318               633      1
2   3.99          2003      15117              3416               398      1
3   3.99          2001      15117              1273               267      1
4   3.99          1999       5659              5250               288      1

In [18]:
X = df[['price', 'release_year', 'developer', 'positive_ratings', 'negative_ratings']]
y = df['label']

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [19]:
# Dicionário para guardar os resultados
results = {}

# 1. Árvore de Decisão
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

results['Decision Tree'] = {
    'accuracy': accuracy_score(y_test, y_pred_dt),
    'report': classification_report(y_test, y_pred_dt, output_dict=True),
    'conf_matrix': confusion_matrix(y_test, y_pred_dt)
}

# 2. Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

results['Random Forest'] = {
    'accuracy': accuracy_score(y_test, y_pred_rf),
    'report': classification_report(y_test, y_pred_rf, output_dict=True),
    'conf_matrix': confusion_matrix(y_test, y_pred_rf)
}

# 3. SVM
svm = SVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

results['SVM'] = {
    'accuracy': accuracy_score(y_test, y_pred_svm),
    'report': classification_report(y_test, y_pred_svm, output_dict=True),
    'conf_matrix': confusion_matrix(y_test, y_pred_svm)
}

# 4. Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)

results['Naive Bayes'] = {
    'accuracy': accuracy_score(y_test, y_pred_nb),
    'report': classification_report(y_test, y_pred_nb, output_dict=True),
    'conf_matrix': confusion_matrix(y_test, y_pred_nb)
}

In [20]:
accuracy_table = pd.DataFrame({
    model: [metrics['accuracy']] for model, metrics in results.items()
}, index=['Accuracy']).T

accuracy_table


Accuracy
Decision Tree  0.992059
Random Forest  0.990028
SVM            0.612188
Naive Bayes    0.576731

In [21]:
plt.figure(figsize=(8, 5))
sns.barplot(x=accuracy_table.index, y='Accuracy', data=accuracy_table.reset_index())
plt.title('Comparação de Acurácia entre os Algoritmos')
plt.ylabel('Acurácia')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
